# SRM-SECT

Magnetic remanence on sections

In [120]:
# force Jupyter to autoreload modules when they have been edited
# this helps keep the iodp module current as it is actively developed.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
import sys
import os
from importlib import reload

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from iodp import utils, srm



In [122]:
# Ensure we are in the Magnetism root folder
if not os.getcwd().endswith("Magnetism"):
    print("Current working directory:", os.getcwd())
    os.chdir("../")
    
print("New current working directory:", os.getcwd())

New current working directory: d:\archive\hd_files\data_analysis\50_laboratory_notebooks\SOD-Laboratory\Magnetism


## Read SRM-SECT .SRM files

In [123]:
file = './data/input/SRM-SECT/402-U1612A-3R-1-A_SHLF13015341_20240215205544__NRM.SRM'

df = utils.read_instrument_file(file, as_dataframe=True)

# NOTE: .SRM files include several sections with measurements taken before the section-half, over the section-half, after the section-half, and drift before/after measurement
# The drift measurements appear at the end of this dataframe.
# The best way to get the dataframe output similar to the .csv output is to sort the whole dataframe on offset. Convert datatypes first.
# The DRIFTS in the data_type column need to be handled manually.
df['offset'] = df['offset'].astype(float)
df = df.sort_values(by='offset', ascending=True)
df.head()


,datetime,labelid,user,text_id,instrument,instrument_group,observed_length,comment,sample_orientation,transform_matrix,...,inclination_w_drift,declination_w_drift,intensity_w_drift,inclination_w_bkgrd_w_drift,declination_w_bkgrd_w_drift,intensity_w_bkgrd_w_drift,tray_position,data_type,Ships heading,track_position
0,2024-02-15 20:55:44 UTC,402-U1612A-3R-1-A,KOWALSKI,SHLF13015341,"0) 2G MODEL: SR760R S/N: 0600, 1) 2G MODEL: SR...",SRM,149.00,,,"transform_matrix = [1.00,0.00,0.00],[0.00,1.00...",...,8.4,37.6,3.698E-5,8.4,37.6,3.698E-5,-14.0,LEADER,269.56,NaN
1,2024-02-15 20:55:44 UTC,402-U1612A-3R-1-A,KOWALSKI,SHLF13015341,"0) 2G MODEL: SR760R S/N: 0600, 1) 2G MODEL: SR...",SRM,149.00,,,"transform_matrix = [1.00,0.00,0.00],[0.00,1.00...",...,2.5,46.7,5.090E-5,2.5,46.7,5.090E-5,-12.0,LEADER,269.58,NaN
2,2024-02-15 20:55:44 UTC,402-U1612A-3R-1-A,KOWALSKI,SHLF13015341,"0) 2G MODEL: SR760R S/N: 0600, 1) 2G MODEL: SR...",SRM,149.00,,,"transform_matrix = [1.00,0.00,0.00],[0.00,1.00...",...,-6.6,55.4,7.171E-5,-6.6,55.4,7.171E-5,-10.0,LEADER,269.66,NaN
3,2024-02-15 20:55:44 UTC,402-U1612A-3R-1-A,KOWALSKI,SHLF13015341,"0) 2G MODEL: SR760R S/N: 0600, 1) 2G MODEL: SR...",SRM,149.00,,,"transform_matrix = [1.00,0.00,0.00],[0.00,1.00...",...,-26.6,68.1,9.582E-5,-26.6,68.1,9.582E-5,-8.0,LEADER,269.66,NaN
4,2024-02-15 20:55:44 UTC,402-U1612A-3R-1-A,KOWALSKI,SHLF13015341,"0) 2G MODEL: SR760R S/N: 0600, 1) 2G MODEL: SR...",SRM,149.00,,,"transform_matrix = [1.00,0.00,0.00],[0.00,1.00...",...,-65.2,136.8,1.711E-4,-65.2,136.8,1.711E-4,-6.0,LEADER,269.61,NaN


In [124]:
# notice the drifts listed at the end:
df.tail()

,datetime,labelid,user,text_id,instrument,instrument_group,observed_length,comment,sample_orientation,transform_matrix,...,inclination_w_drift,declination_w_drift,intensity_w_drift,inclination_w_bkgrd_w_drift,declination_w_bkgrd_w_drift,intensity_w_bkgrd_w_drift,tray_position,data_type,Ships heading,track_position
83,2024-02-15 20:55:44 UTC,402-U1612A-3R-1-A,KOWALSKI,SHLF13015341,"0) 2G MODEL: SR760R S/N: 0600, 1) 2G MODEL: SR...",SRM,149.00,,,"transform_matrix = [1.00,0.00,0.00],[0.00,1.00...",...,37.9,329.4,2.784E-5,37.9,329.4,2.784E-5,158.0,TRAILER,269.70,NaN
84,2024-02-15 20:55:44 UTC,402-U1612A-3R-1-A,KOWALSKI,SHLF13015341,"0) 2G MODEL: SR760R S/N: 0600, 1) 2G MODEL: SR...",SRM,149.00,,,"transform_matrix = [1.00,0.00,0.00],[0.00,1.00...",...,39.3,341.8,2.187E-5,39.3,341.8,2.187E-5,160.0,TRAILER,269.70,NaN
85,2024-02-15 20:55:44 UTC,402-U1612A-3R-1-A,KOWALSKI,SHLF13015341,"0) 2G MODEL: SR760R S/N: 0600, 1) 2G MODEL: SR...",SRM,149.00,,,"transform_matrix = [1.00,0.00,0.00],[0.00,1.00...",...,36.5,347.1,1.984E-5,36.5,347.1,1.984E-5,162.0,TRAILER,269.72,NaN
86,2024-02-15 20:55:44 UTC,402-U1612A-3R-1-A,KOWALSKI,SHLF13015341,"0) 2G MODEL: SR760R S/N: 0600, 1) 2G MODEL: SR...",SRM,149.00,,,"transform_matrix = [1.00,0.00,0.00],[0.00,1.00...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DRIFT #1,269.6,326.0
87,2024-02-15 20:55:44 UTC,402-U1612A-3R-1-A,KOWALSKI,SHLF13015341,"0) 2G MODEL: SR760R S/N: 0600, 1) 2G MODEL: SR...",SRM,149.00,,,"transform_matrix = [1.00,0.00,0.00],[0.00,1.00...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DRIFT #2,269.7,106.0


In [ ]:
# Alternatively, you can also access portions using the dictionary approach:
srm_dict = utils.read_instrument_file(file, as_dataframe=False)
print(srm_dict.keys())
pd.DataFrame(srm_dict['MULTI-DRIFT'])

dict_keys(['datetime', 'labelid', 'user', 'text_id', 'instrument', 'instrument_group', 'observed_length', 'comment', 'sample_orientation', 'transform_matrix', 'transform_comment', 'sample_shape', 'sample_area', 'daq_interval', 'daq_iterations', 'leader-trailer_length', 'treatment_comment', 'measurement_type', 'MULTI-LEADER', 'MULTI', 'MULTI-TRAILER', 'MULTI-DRIFT', 'file_srm_section_bkgnd', 'file_raw', 'file_srm_sequence', 'file_configuration', 'positions_excluded'])


,track_position,meas_time,x_meas_moment,y_meas_moment,z_meas_moment,data_type,Ships heading
0,326.0,3790875207.945062,2.865E-8,-4.661E-8,1.784E-8+0,DRIFT #1,269.6
1,106.0,3790875344.652421,1.840E-10,1.900E-10,1.290E-9+0,DRIFT #2,269.7


In [126]:
pd.DataFrame(srm_dict['MULTI']).head()

,offset,treatment_type,treatment_value,meas_time,x_meas_moment,y_meas_moment,z_meas_moment,inclination_w_raw,declination_w_raw,intensity_w_raw,...,intensity_w_bkgrd,inclination_w_drift,declination_w_drift,intensity_w_drift,inclination_w_bkgrd_w_drift,declination_w_bkgrd_w_drift,intensity_w_bkgrd_w_drift,tray_position,data_type,Ships heading
0,0.0,NRM,0,3790875221.004284,-7.407E-8,5.659E-8,-1.579E-7,-54.0,142.6,1.240E-3,...,1.241E-3,-54.6,136.2,1.189E-3,-54.6,136.2,1.189E-3,0.0,SAMPLE,269.71
1,2.0,NRM,0,3790875222.537177,-7.386E-7,-8.431E-7,-3.473E-7,-14.1,228.8,9.047E-3,...,9.047E-3,-14.0,229.0,8.989E-3,-14.0,229.0,8.989E-3,2.0,SAMPLE,269.70
2,4.0,NRM,0,3790875224.074166,-1.201E-6,-2.226E-6,-9.815E-7,-17.5,241.7,2.076E-2,...,2.076E-2,-17.4,241.7,2.073E-2,-17.4,241.7,2.073E-2,4.0,SAMPLE,269.74
3,6.0,NRM,0,3790875225.604336,-2.100E-8,-2.144E-6,-1.793E-6,-34.1,269.4,2.028E-2,...,2.028E-2,-34.1,269.5,2.027E-2,-34.1,269.5,2.027E-2,6.0,SAMPLE,269.65
4,8.0,NRM,0,3790875227.149235,1.314E-6,-1.124E-6,-1.360E-6,-32.5,319.5,1.606E-2,...,1.606E-2,-32.5,319.5,1.606E-2,-32.5,319.5,1.606E-2,8.0,SAMPLE,269.60


# Read SRM-SECT settings .ini file

In [127]:
file = './data/input/SRM-SECT/i_pi_srm.ini'
df = utils.read_instrument_ini(file,as_dataframe=True)
df

,value
key,
mgi rwa section options,2.0.1 %04Y%02m%02d %02H%02M%S%25u*~|.%d*~|....
ims framework-major,14
ims framework-minor,0
plugin-major,0
plugin-minor,0
...,...
discrete.raw_w_drift 6..treatment,Cancel
discrete.raw_w_drift 6..demag/t,0
discrete.raw_w_drift 6..hdg,81.45664699987


# Read SRM-SECT sequence .ini file

In [128]:
file = './data/input/SRM-SECT/nrm only 20231011173915_seq.ini'
df = utils.read_instrument_ini(file, as_dataframe=True)
df


,value
key,
mgi rwa section options,2.0.1 %04Y%02m%02d %02H%02M%S%25u*~|.%d*~|....
path,C:\IMS\CONFIG_SRM\User Sequence Files\NRM Only...
name,NRM Only 20231011173915_SEQ.ini
actions,<1>
actions 0.system cluster.seq step type,Measure Only
actions 0.system cluster.field gauss,0
actions 0.system cluster.before saving...,Do Nothingline*~|.feedJust Save Automatically
actions 0.system cluster.offline.offline treatment,Cancel
actions 0.system cluster.offline.axis used.x,TRUE


# Read SRM measurement .csv file

In [129]:
file = './data/input/SRM-SECT/402-u1612a-3r-1-a_shlf13015341_20240215205544__nrm.csv'
df = srm.read_srm_csv(file)
df


,Sample ID,Text_ID,Offset,Depth,Heading,Mode,Type,Shape,Shape Value,TimeStamp,...,Y-Flux-Remaing,Z-Flux-Remaing,X-SQUID Calib Constant,X-SQUID Response Length,Y-SQUID Calib Constant,Y-SQUID Response Length,Z-SQUID Calib Constant,Z-SQUID Response Length,Section Tray Length,Top Offset
0,,,326.0,,,,Drift #1,,,3790875207.945062,...,,,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
1,402-U1612A-3R-1-A,SHLF13015341,-14.0,,,,Leader,,,3790875210.887267,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
2,402-U1612A-3R-1-A,SHLF13015341,-12.0,,,,Leader,,,3790875212.301450,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
3,402-U1612A-3R-1-A,SHLF13015341,-10.0,,,,Leader,,,3790875213.706722,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
4,402-U1612A-3R-1-A,SHLF13015341,-8.0,,,,Leader,,,3790875215.112998,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,402-U1612A-3R-1-A,SHLF13015341,158.0,,,,Trailer,,,3790875337.737837,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
85,402-U1612A-3R-1-A,SHLF13015341,160.0,,,,Trailer,,,3790875339.142111,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
86,402-U1612A-3R-1-A,SHLF13015341,162.0,,,,Trailer,,,3790875340.554371,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
87,,,106.0,,,,Drift #2,,,3790875344.652421,...,,,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00


# Read SRM section-tray background .csv file

In [130]:
file = './data/input/SRM-SECT/section-tray_20240215204550__nrm.csv'
df = srm.read_srm_csv(file)
df

,Sample ID,Text_ID,Offset,Depth,Heading,Mode,Type,Shape,Shape Value,TimeStamp,...,Y-Flux-Remaing,Z-Flux-Remaing,X-SQUID Calib Constant,X-SQUID Response Length,Y-SQUID Calib Constant,Y-SQUID Response Length,Z-SQUID Calib Constant,Z-SQUID Response Length,Section Tray Length,Top Offset
0,,,326.0,,,,Drift #1,,,3790874085.644541,...,,,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
1,BKGND,SECTION TRAY,-14.0,,,,Leader,,,3790874089.231180,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
2,BKGND,SECTION TRAY,-12.0,,,,Leader,,,3790874091.278890,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
3,BKGND,SECTION TRAY,-10.0,,,,Leader,,,3790874093.334426,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
4,BKGND,SECTION TRAY,-8.0,,,,Leader,,,3790874095.375001,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,BKGND,SECTION TRAY,170.0,,,,Trailer,,,3790874275.727743,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
94,BKGND,SECTION TRAY,172.0,,,,Trailer,,,3790874277.779303,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
95,BKGND,SECTION TRAY,174.0,,,,Trailer,,,3790874279.824864,...,0,0,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
96,,,106.0,,,,Drift #2,,,3790874283.148662,...,,,8.1036E-5,7.300,-8.2590E-5,7.300,3.8825E-5,9.000,160.0,2.00
